# Importing some libraries

In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

# GPIO Library Functions

In [100]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"
#define GPIO0 0
#define GPIO1 1
#define GPIO2 2

//Function to turn on/off a selected pin of PMODA
void write_gpio(unsigned int pin, unsigned int val)
{
    if (val > 1)
    {
        pyprintf("pin value must be 0 or 1");
    }
    
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODA
unsigned int read_gpio(unsigned int pin)
{
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    int a = gpio_read(pin_in);
    pyprintf("pin %d --- val is %d\n", pin, a);
    return a;    
}

void reset_all_gpios()
{
    pyprintf("reset PMOD \n");
    for(int pin=0; pin < 6; pin++) 
    {
        write_gpio(pin, 0);
    }    
}

# Emulate a Pulse Width Modulation

In [101]:
import time

def emulatePwm(frequency, dutycycle, color):
    period = 1 / frequency
    newDutyCycle = dutycycle / 100
    delay = period * newDutyCycle
    
    #print("on delay: ", delay)
    #print("off delay:", period - delay)
    count = frequency
    
    while count:
        write_gpio(0, 0x1);
        time.sleep(delay);
        read_gpio(0);
        
        
        write_gpio(0, 0x0);
        time.sleep(period - delay);
    
        count -= 1

# Main Loop

In [103]:
reset_all_gpios()

i = 1

while i < 1000:
    emulatePwm(1, 20, 4)
    i = i+1

reset PMOD 
pin 0 --- val is 1
pin 0 --- val is 1
pin 0 --- val is 1
pin 0 --- val is 1
pin 0 --- val is 1
pin 0 --- val is 1
pin 0 --- val is 1
pin 0 --- val is 1
pin 0 --- val is 1
pin 0 --- val is 1
pin 0 --- val is 1


KeyboardInterrupt: 

# Testing LEDS

In [22]:
noColor = 0x0
blue = 0x1
green = 0x2
red = 0x4

In [5]:
base.leds[0].on()

In [205]:
base.leds[0].off()

In [23]:
import pynq.lib.rgbled as rgbled 
led4 = rgbled.RGBLED(4)

In [17]:
led4.write(green)

In [15]:
led4.write(blue)

In [23]:
led4.write(red)

In [33]:
led4.off()

# Testing Buttons

In [109]:
import asyncio
import pynq.lib.rgbled as rgbled 
led4 = rgbled.RGBLED(4)
btns = base.btns_gpio

In [110]:
class Buttons:    
    none = 0
    BTN0 = 1
    BTN1 = 2
    
noColor = 0x0
blue = 0x1
green = 0x2
red = 0x4
    
numColorChanges = 0
maxColorChanges = 4
lastBtnPressed = Buttons.none
buttonPressed = Buttons.none

checkBtnsSleepTime = 0.01

blinking = True

In [111]:
def lightLed(color):
    global numColorChanges, lastBtnPressed, buttonPressed
    
    if (lastBtnPressed != buttonPressed) and (buttonPressed != Buttons.none):
        led4.write(color)
        numColorChanges += 1
        lastBtnPressed = buttonPressed

In [112]:
async def checkButtons(_loop):
    global numColorChanges, maxColorChanges, lastBtnPressed, buttonPressed, checkBtnsSleepTime
    
    numColorChanges = 0
    maxColorChanges = 4
    lastBtnPressed = Buttons.none
    buttonPressed = Buttons.none

    led4.off()
    base.leds[0].on()
    base.leds[1].on()

    while numColorChanges < maxColorChanges:
        await asyncio.sleep(checkBtnsSleepTime)
        buttonPressed = btns.read()

        match buttonPressed:          
            case Buttons.BTN0:
                lightLed(green)
            case Buttons.BTN1:
                lightLed(red)
                
    led4.off()
    base.leds[0].off()
    base.leds[1].off()
    base.leds[3].off()
    blinking = False
    _loop.stop()

In [113]:
async def transmitClicking():
    global blinking, lastBtnPressed
    blinkSleepTime = 0.3
    
    base.leds[3].off()
    reset_all_gpios()
    
    while blinking:
        await asyncio.sleep(blinkSleepTime)
        
        if lastBtnPressed == Buttons.BTN0:
            blinkSleepTime = 0.3
            write_gpio(0, 0x1);
            base.leds[3].on()
            await asyncio.sleep(blinkSleepTime)
            write_gpio(0, 0x0);
            base.leds[3].off()
            
        if lastBtnPressed == Buttons.BTN1:
            blinkSleepTime = 0.1
            write_gpio(0, 0x1);
            base.leds[3].on()
            await asyncio.sleep(blinkSleepTime)
            write_gpio(0, 0x0);
            base.leds[3].off()

In [114]:
loop = asyncio.new_event_loop()
loop.create_task(transmitClicking())
loop.create_task(checkButtons(loop))
loop.run_forever()
loop.close()  

print("Finished.\n")

Finished.

